In [24]:
from bs4 import BeautifulSoup
from openpyxl import load_workbook, Workbook
import random
import requests
import string

In [11]:
#“Giełda”, “Linki” i “Filmweb”
wb = Workbook()
wb.active.title = "Giełda"
wb.create_sheet("Linki")
wb.create_sheet("Filmweb")

print(wb.sheetnames)

['Giełda', 'Linki', 'Filmweb']


In [28]:
# 5 losowych kodów 3-literowych...

codes = []
url = "https://stooq.pl/q/?s="
while len(codes)!=5:
    code=''.join([random.choice(string.ascii_lowercase) for i in range(3)])
    new_url = url+code
    soup = BeautifulSoup(requests.get(new_url).text,"html.parser")
    if "Czy chodziło ci o" not in soup.get_text():
        codes.append(code)
        
codes

['dds', 'lrt', 'gag', 'isb', 'cfu']

In [20]:
# Aktualna cena, procentowa zmiana oraz liczba transakcji...







<html><head><meta content="text/html;charset=utf-8" http-equiv="content-type"/><meta content="symbole, wykresy, znajdÅº symbol, wyszukaj symbol" name="keywords"/><meta content="Wyszukiwanie symbolu" name="description"/><title>Wyszukiwanie symbolu - Stooq</title><script>var $=function(s){return document.getElementById(s)};!function(n,e,o){function t(n,e){return typeof n===e}function s(){var n,e,o,s,a,f,l;for(var c in i)if(i.hasOwnProperty(c)){if(n=[],e=i[c],e.name&&(n.push(e.name.toLowerCase()),e.options&&e.options.aliases&&e.options.aliases.length))for(o=0;o<e.options.aliases.length;o++)n.push(e.options.aliases[o].toLowerCase());for(s=t(e.fn,"function")?e.fn():e.fn,a=0;a<n.length;a++)f=n[a],l=f.split("."),1===l.length?Modernizr[l[0]]=s:(!Modernizr[l[0]]||Modernizr[l[0]]instanceof Boolean||(Modernizr[l[0]]=new Boolean(Modernizr[l[0]])),Modernizr[l[0]][l[1]]=s),r.push((s?"":"no-")+l.join("-"))}}var i=[],a={_version:"3.5.0",_config:{classPrefix:"",enableClasses:!0,enableJSClass:!0,usePref